In [4]:
# 시뮬레이션 라이브러리 불러오기
import meep as mp
import meep.adjoint as mpa
import numpy as np
from autograd import numpy as npa
from autograd import tensor_jacobian_product, grad
import nlopt
from meep_material import lorentzfit
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import pickle
import math
import os
import sys

In [5]:
def Material_fit(Material_data_csv=".",
                 eps_inf_min=1, eps_inf_max=1.2, eps_inf_linespace=1,
                 wl_min=0.39, wl_max=0.71,
                 num_L=3, num_rep=100):
    # --- 0) 피클 경로 설정 ---
    base, _ = os.path.splitext(Material_data_csv)
    pickle_path = base + "_fit.pkl"

    # --- 1) 피클이 있으면 로드 후 반환 ---
    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as f:
            eps_inf_best, suscepts = pickle.load(f)
        print(f"Loaded fit from pickle: {pickle_path}")
        return eps_inf_best, suscepts

    # --- 2) CSV 로드 및 준비 ---
    data      = np.genfromtxt(Material_data_csv, delimiter=",")
    wl        = data[:, 0]                    # 파장 (μm)
    eps_total = data[:, 1] + 1j*data[:, 2]     # permittivity

    mask     = (wl >= wl_min) & (wl <= wl_max)
    wl_red   = wl[mask]
    freqs_red = 1.0 / wl_red
    eps_total_red = eps_total[mask]
    eps_fit_target_raw = eps_total_red        # eps_inf 뺄 전 원본

    # --- 3) 내부 피팅 함수 ---
    def fit_for_eps_inf(eps_inf):
        eps_fit_target = eps_fit_target_raw - eps_inf
        best_err = np.inf
        best_p   = None
        for _ in range(num_rep):
            p0 = []
            for _ in range(num_L):
                # 초기 파라미터: σ∈[1,100], ω∈[1/λ_max,1/λ_min], γ∈[1e-2,1]
                sigma0 = 10**np.random.uniform(0, 2)
                omega0 = np.random.uniform(1.0/wl_max, 1.0/wl_min)
                gamma0 = 10**np.random.uniform(-2, 0)
                p0.extend([sigma0, omega0, gamma0])
            p_opt, err = lorentzfit(p0, freqs_red, eps_fit_target,
                                    nlopt.LD_MMA, 1e-25, 200000)
            if err < best_err:
                best_err = err
                best_p   = p_opt
        return best_err, best_p

    # --- 4) eps_inf 스윕 및 최적화 ---
    eps_inf_vals = np.linspace(eps_inf_min, eps_inf_max, eps_inf_linespace)
    results = []
    for e_inf in eps_inf_vals:
        err, p_opt = fit_for_eps_inf(e_inf)
        results.append((e_inf, err, p_opt))

    eps_inf_best, err_best, p_opt_best = min(results, key=lambda x: x[1])
    print("Best eps_inf:", eps_inf_best, "error:", err_best)

    # --- 5) susceptibilities 생성 ---
    suscepts = []
    for j in range(num_L):
        ω = p_opt_best[3*j + 1]
        γ = p_opt_best[3*j + 2]
        if abs(ω) < 1e-8:
            σ = p_opt_best[3*j + 0]
            suscepts.append(mp.DrudeSusceptibility(frequency=1.0, gamma=γ, sigma=σ))
        else:
            σ = p_opt_best[3*j + 0] / ω**2
            suscepts.append(mp.LorentzianSusceptibility(frequency=ω, gamma=γ, sigma=σ))

    # --- 6) 결과를 피클로 저장 ---
    with open(pickle_path, "wb") as f:
        pickle.dump((eps_inf_best, suscepts), f)
    print(f"Saved fit to pickle: {pickle_path}")

    return eps_inf_best, suscepts

sio2_data_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/lumerical_sio2_eps.csv"
al_data_path   = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/lumerical_al_eps.csv"
andp_data_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/lumerical_aNDP_eps.csv"

eps_inf_sio2, suscept_sio2 = Material_fit(Material_data_csv=sio2_data_path)
eps_inf_al, suscept_al = Material_fit(Material_data_csv=al_data_path)
eps_inf_andp, suscept_andp = Material_fit(Material_data_csv=andp_data_path)

Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/lumerical_sio2_eps_fit.pkl
Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/lumerical_al_eps_fit.pkl
Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/lumerical_aNDP_eps_fit.pkl


In [7]:
### ----- Refractive index ----- ###,
sio2 =  mp.Medium(epsilon = eps_inf_sio2,   E_susceptibilities = suscept_sio2) # SiO2 ,
andp =  mp.Medium(epsilon = eps_inf_andp,   E_susceptibilities = suscept_andp) # aNDP ,
Al  =   mp.Medium(epsilon = eps_inf_al,     E_susceptibilities = suscept_al)
Air =   mp.Medium(index = 1)

In [12]:
resolution = 1000 # 해상도
design_region_x = 0.01 # 디자인 영역 x
design_region_y = 0.01 # 디자인 영역 y
design_region_z = 2 # 디자인 영역 높이 z
pml_size = 1.0 # PML 영역 크기

In [13]:
# 시뮬레이션 공간 설정
Sx = design_region_x
Sy = design_region_y
Sz = 2 * pml_size + design_region_z + 1 + 5
cell_size = mp.Vector3(0, 0, Sz)

In [58]:
from pathlib import Path

al_csv_path = Path("/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/Al/Material data/al_material_data.csv")
sio2_csv_path = Path("/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/SiO2/Material data/sio2_material_data.csv")
andp_csv_path = Path("/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data.csv")

# Wavelength data
sio2_data = np.genfromtxt(sio2_csv_path, delimiter=",")
andp_data = np.genfromtxt(andp_csv_path, delimiter=",")
al_data = np.genfromtxt(al_csv_path, delimiter=",")

sio2_wavelengths = sio2_data[:,0]
andp_wavelengths = andp_data[:,0]
al_wavelengths = al_data[:,0]

sio2_frequencies = 1 / sio2_wavelengths
andp_frequencies = 1 / andp_wavelengths
al_frequencies = 1 / al_wavelengths

nf_sio2 = len(sio2_frequencies)
nf_andp = len(andp_frequencies)
nf_al = len(al_frequencies)

In [33]:
minimum_length = 0.05  # minimum length scale (microns)
eta_i = 0.5  # blueprint (or intermediate) design field thresholding point (between 0 and 1)
eta_e = 0.55  # erosion design field thresholding point (between 0 and 1)
eta_d = 1 - eta_e  # dilation design field thresholding point (between 0 and 1)
filter_radius = mpa.get_conic_radius_from_eta_e(minimum_length, eta_e)
design_region_resolution = int(resolution)

pml_layers = [mp.PML(thickness = pml_size, direction = mp.Z)]

In [65]:
materials = {
    'al':   {'wavelengths': al_data[:,0],   'frequencies': 1 / al_data[:,0]},
    'sio2': {'wavelengths': sio2_data[:,0], 'frequencies': 1 / sio2_data[:,0]},
    'andp': {'wavelengths': andp_data[:,0], 'frequencies': 1 / andp_data[:,0]},
}

# --- 2) 색상별 파장 범위 정의 ---
wl_ranges = {
    'blue':  (0.4, 0.50),
    'green': (0.50, 0.60),
    'red':   (0.60, 0.7),
}

width = 0.1  # fwidth 계산용 계수

# --- 3) 재료별로 색상별 freqs/fwidths 및 총 필터된 포인트 계산 ---
results = {}

for mat, data in materials.items():
    wl = data['wavelengths']
    fr = data['frequencies']
    total_count = 0
    freqs_color = {}
    fwidths_color = {}
    for color, (wl_min, wl_max) in wl_ranges.items():
        mask = (wl >= wl_min) & (wl <= wl_max)
        freqs_color[color] = fr[mask]
        fwidths_color[color] = width * fr[mask]
        total_count += np.count_nonzero(mask)
    results[mat] = {
        'freqs': freqs_color,
        'fwidths': fwidths_color,
        'total_filtered_points': total_count
    }

# --- 4) 결과 출력 ---
for mat, data in results.items():
    print(f"\n== {mat.upper()} ==")
    for color in wl_ranges:
        print(f"{color.title():6s} frequencies:", data['freqs'][color])
        print(f"{color.title():6s} fwidths:   ", data['fwidths'][color])
    print(f"Total filtered points: {data['total_filtered_points']}")


== AL ==
Blue   frequencies: [2.         2.01653559 2.09687566 2.17770035 2.22222222 2.25835592
 2.33918129 2.41954996 2.5       ]
Blue   fwidths:    [0.2        0.20165356 0.20968757 0.21777003 0.22222222 0.22583559
 0.23391813 0.241955   0.25      ]
Green  frequencies: [1.66666667 1.69376694 1.73400381 1.77430802 1.81488203 1.81818182
 1.85494342 1.89537528 1.93573364 1.97589409 2.        ]
Green  fwidths:    [0.16666667 0.16937669 0.17340038 0.1774308  0.1814882  0.18181818
 0.18549434 0.18953753 0.19357336 0.19758941 0.2       ]
Red    frequencies: [1.42857143 1.45180023 1.49209191 1.53233221 1.53846154 1.57282164
 1.61316341 1.65343915 1.66666667]
Red    fwidths:    [0.14285714 0.14518002 0.14920919 0.15323322 0.15384615 0.15728216
 0.16131634 0.16534392 0.16666667]
Total filtered points: 29

== SIO2 ==
Blue   frequencies: [2.05705023 2.13759256 2.29444629 2.47123483]
Blue   fwidths:    [0.20570502 0.21375926 0.22944463 0.24712348]
Green  frequencies: [1.69703799 1.70195095 1.726

In [68]:
# al - frequency, fwidth 결과 슬라이싱  
al_red_freqs = results['al']['freqs']['red']
al_green_freqs = results['al']['freqs']['green']
al_blue_freqs = results['al']['freqs']['blue']

al_red_fwidths = results['al']['fwidths']['red']
al_green_fwidths = results['al']['fwidths']['green']
al_blue_fwidths = results['al']['fwidths']['blue']

# aNDP - frequency, fwidth 결과 슬라이싱  
andp_red_freqs = results['andp']['freqs']['red']
andp_green_freqs = results['andp']['freqs']['green']
andp_blue_freqs = results['andp']['freqs']['blue']

andp_red_fwidths = results['andp']['fwidths']['red']
andp_green_fwidths = results['andp']['fwidths']['green']
andp_blue_fwidths = results['andp']['fwidths']['blue']

# sio2 - frequency, fwidth 결과 슬라이싱  
sio2_red_freqs = results['sio2']['freqs']['red']
sio2_green_freqs = results['sio2']['freqs']['green']
sio2_blue_freqs = results['sio2']['freqs']['blue']

sio2_red_fwidths = results['sio2']['fwidths']['red']
sio2_green_fwidths = results['sio2']['fwidths']['green']
sio2_blue_fwidths = results['sio2']['fwidths']['blue']

In [71]:
source_center = [0, 0, Sz / 2 - pml_size] # Source 위치

src_0 = mp.GaussianSource(frequency=al_red_freqs[1], fwidth=al_red_fwidths[1], is_integrated=True)


source = [mp.Source(src_0, component=mp.Ey, center=source_center)]


In [73]:
Nx = int(round(design_region_resolution * design_region_x)) + 1
Ny = int(round(design_region_resolution * design_region_y)) + 1
Nz = int(round(design_region_resolution * design_region_z)) + 1

design_variables = mp.MaterialGrid(mp.Vector3(0, 0, Nz), Air, andp, grid_type="U_MEAN")

In [74]:
design_region = mpa.DesignRegion(
    design_variables,
    volume=mp.Volume(
        center=mp.Vector3(0, 0, Sz / 2 - pml_size - 2),
        size=mp.Vector3(design_region_x, design_region_y, design_region_z),
    ),
)


In [76]:
geometry = [
    mp.Block(
        center=design_region.center, size=mp.Vector3(mp.inf, mp.inf, 0.2), material=andp
    )
]

In [78]:
sim = mp.Simulation(
    cell_size=cell_size, 
    boundary_layers=pml_layers,
    geometry=geometry,
    sources=source,
    default_material=Air, # 빈공간
    resolution=resolution,
    extra_materials=[andp],
    k_point = mp.Vector3(0,0,0),
    dimensions=1
)

# 모니터 위치와 크기 설정 (focal point)
monitor_position_0, monitor_size_0 = mp.Vector3(-Sx/2 + 1, Sy/2  - 1, -Sz/2 + pml_size + 0.5/resolution), mp.Vector3(0.01,0.01,0) 
monitor_position_1, monitor_size_1 = mp.Vector3(-Sx/2 + 1, -Sy/2 + 1, -Sz/2 + pml_size + 0.5/resolution), mp.Vector3(0.01,0.01,0) 
monitor_position_2, monitor_size_2 = mp.Vector3(Sx/2 - 1, -Sy/2 + 1, -Sz/2 + pml_size + 0.5/resolution), mp.Vector3(0.01,0.01,0) 
monitor_position_3, monitor_size_3 = mp.Vector3(Sx/2 - 1, Sy/2  -  1, -Sz/2 + pml_size + 0.5/resolution), mp.Vector3(0.01,0.01,0) 

# FourierFields를 통해 monitor_position에서 monitor_size만큼의 영역에 대한 Fourier transform을 구함
FourierFields_0 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_0,size=monitor_size_0),mp.Ey,yee_grid=True)
FourierFields_1 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_0,size=monitor_size_0),mp.Ey,yee_grid=True)
FourierFields_2 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_0,size=monitor_size_0),mp.Ey,yee_grid=True)
FourierFields_3 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_1,size=monitor_size_1),mp.Ey,yee_grid=True)
FourierFields_4 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_1,size=monitor_size_1),mp.Ey,yee_grid=True)
FourierFields_5 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_1,size=monitor_size_1),mp.Ey,yee_grid=True)
FourierFields_6 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_2,size=monitor_size_2),mp.Ey,yee_grid=True)
FourierFields_7 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_2,size=monitor_size_2),mp.Ey,yee_grid=True)
FourierFields_8 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_2,size=monitor_size_2),mp.Ey,yee_grid=True)
FourierFields_9 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_3,size=monitor_size_3),mp.Ey,yee_grid=True)
FourierFields_10 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_3,size=monitor_size_3),mp.Ey,yee_grid=True)
FourierFields_11 = mpa.FourierFields(sim,mp.Volume(center=monitor_position_3,size=monitor_size_3),mp.Ey,yee_grid=True)
ob_list = [FourierFields_0, FourierFields_1, FourierFields_2,FourierFields_3, FourierFields_4, FourierFields_5 ,FourierFields_6, FourierFields_7, FourierFields_8,FourierFields_9, FourierFields_10, FourierFields_11]

# In[16]:


# J : Objective function
# FourierFields가 측정한 필드, 모니터의 중심에서 Ez 구성요소의 절댓값을 제곱한 값을 취한 후 평균을 계산하여 평균 강도를 계산
def J_0(fields_0, fields_1, fields_2,fields_3, fields_4, fields_5,fields_6, fields_7, fields_8,fields_9, fields_10, fields_11):
    return npa.mean(npa.abs(fields_0[6,:]) ** 2) + npa.mean(npa.abs(fields_1[7,:]) ** 2) + npa.mean(npa.abs(fields_2[8,:]) ** 2) + npa.mean(npa.abs(fields_3[3,:]) ** 2) + npa.mean(npa.abs(fields_4[4,:]) ** 2) + npa.mean(npa.abs(fields_5[5,:]) ** 2) + npa.mean(npa.abs(fields_6[0,:]) ** 2) + npa.mean(npa.abs(fields_7[1,:]) ** 2) + npa.mean(npa.abs(fields_8[2,:]) ** 2) + npa.mean(npa.abs(fields_9[3,:]) ** 2) + npa.mean(npa.abs(fields_10[4,:]) ** 2) + npa.mean(npa.abs(fields_11[5,:]) ** 2)

opt = mpa.OptimizationProblem(
    simulation=sim,
    objective_functions=[J_0],
    objective_arguments=ob_list,
    design_regions=[design_region],
    frequencies=frequencies,
    decay_by=1e-9
)
